# init

In [ ]:
import sys, os
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString, MultiLineString, base as sg
from matplotlib import pyplot as plt, axes
from timeit import default_timer as timer
from datetime import timedelta
from pprint import pprint


from flat_norm_fixture import FlatNormFixture


fx = FlatNormFixture('runTest')


# plot regions

## sample_regions

In [ ]:
area = 'mcbryde'
epsilon = 2e-3
num_regions = 6
region = 2

# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

# sample regions
point_list, region_list = fx.sample_regions(
    hull, num_regions=num_regions, epsilon=epsilon, seed=54321
)

# plot regions
fx.fig_dir = "figs/notebooks"
fig, ax = fx.plot_regions_list(
    act_geom, synt_geom, region_list, area,
    show=True,
    region_highlight=region,
    file_name_sfx=f"sr_region_{region}",
    figsize=(40, 20),
    do_return=True
)

## sample_regions_geom

In [ ]:
area = 'mcbryde'
epsilon = 2e-3
num_regions = 6
region = 2

# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

# sample regions
point_list, region_list = fx.sample_regions_geom(
    hull, act_geom, synt_geom,
    num_regions=num_regions, epsilon=epsilon,
    seed=54321
)

# plot regions
fx.fig_dir = "figs/notebooks"
fig, ax = fx.plot_regions_list(
    act_geom, synt_geom, region_list, area,
    show=True,
    region_highlight=region,
    file_name_sfx=f"srgeom_region_{region}",
    figsize=(40, 20),
    do_return=True
)

# compute_and_plot_region_flatnorm

In [ ]:
area = 'mcbryde'
epsilon, lambda_ = 2e-3, 1000
num_regions = 6
region = 2
fx.seed=54321

# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

# sample regions
point_list, region_list = fx.sample_regions(
    hull, num_regions=num_regions, epsilon=epsilon, seed=fx.seed
)

# plot regions
fx.fig_dir = "figs/notebooks"
fx.plot_regions_list(
    act_geom, synt_geom, region_list, area,
    region_highlight=region,
    file_name_sfx=f"sg_region_{region}",
    do_return=False,
    figsize=(40, 20)
)

# flat norm
norm, enorm, tnorm, w, plot_data = fx.compute_region_flatnorm(
    region_list[region], act_geom, synt_geom,
    lambda_=lambda_,
    normalized=True,
    plot=True
)

# plot flat norm
fx.plot_triangulated_region_flatnorm(
    epsilon=epsilon, lambda_=lambda_,
    to_file=f"{area}-flatnorm_region_{region}",
    suptitle_sfx=f"FN={norm:0.5f} : |T| = {w:0.5f} : |T| / $\\epsilon$ = {w / epsilon:0.5f}",
    do_return=False, show=True,
    **plot_data
)


# plot region flatnorm lines

In [ ]:
area = 'mcbryde'
epsilons, lambdas = np.linspace(5e-4, 2e-3, 4), np.linspace(1000, 100000, 5)
num_regions = 6
region = 2
fx.seed=54321

# read geometries
act_geom, synt_geom, hull = fx.read_networks(area)

points, regions = fx.sample_regions_geom(
    hull, act_geom, synt_geom,
    epsilon=epsilons[0],
    num_regions=num_regions,
    seed=fx.seed
)

# plot regions
fx.fig_dir = "figs/test"
fx.plot_regions_list(
    act_geom, synt_geom, regions, area,
    region_highlight=region,
    file_name_sfx=f"_region_{region}",
    do_return=False, show=True,
    figsize=(40, 20)
)

# flat norm
flatnorm_data = {
    'epsilons': [], 'lambdas': [], 'flatnorms': [],
    'norm_lengths': [], 'norm_areas': []
}

start = timer()
for epsilon in epsilons:
    for lambda_ in lambdas:
        norm, enorm, tnorm, w = fx.compute_region_flatnorm(
            fx.get_region(points[region], epsilon),
            act_geom, synt_geom,
            lambda_=lambda_,
            normalized=True,
            plot=False
        )
        flatnorm_data['epsilons'].append(f"{epsilon:0.4f}")
        flatnorm_data['lambdas'].append(lambda_)
        flatnorm_data['flatnorms'].append(norm)
        flatnorm_data['norm_lengths'].append(enorm)
        flatnorm_data['norm_areas'].append(tnorm)

end = timer()

fig, ax = fx.plot_region_flatnorm_lines(
    epsilons=flatnorm_data['epsilons'],
    lambdas=flatnorm_data['lambdas'],
    flatnorms=flatnorm_data['flatnorms'],
    to_file=f"{area}-flatnorm-lines_region_{region}",
    do_return=True
)

print("--------------------------------------------------------------------------")
print(f"compute region flatnorm "
      f"for {len(epsilons)} epsilons "
      f"and {len(lambdas)} lambdas = {timedelta(seconds=end-start)}")
print("--------------------------------------------------------------------------")
flatnorm_data = pd.DataFrame(flatnorm_data)
flatnorm_data


# flatnorm_stats

In [ ]:
fx.out_dir = "out/test"
%ls {fx.out_dir}

In [ ]:
area = 'mcbryde'
epsilons, lambdas = np.linspace(5e-4, 2e-3, 4), np.linspace(1000, 100000, 5)
num_regions = 50
fx.out_dir = "out/test"
file_name = f"{area}-flatnorm-stats_{num_regions}_regions"

flatnorm_df = pd.read_csv(
    f"{fx.out_dir}/{file_name}.csv",
    sep=",",
    # header=
)

In [ ]:
flatnorm_df

In [ ]:
flatnorm_df = flatnorm_df.set_index(['epsilons', 'lambdas'], drop=False)

In [ ]:
plt.hist(flatnorm_df.loc[(epsilons[2], lambdas[2]), 'flatnorms'], bins = 20)


In [ ]:
plt.scatter(
    flatnorm_df.loc[(epsilons[0], lambdas[0]), 'flatnorms'],
    flatnorm_df.loc[(epsilons[0], lambdas[0]), 'input_ratios'],
)
# T  = T1 - T2
# T_ratio = |T| / eps

In [ ]:
fig, axs = plt.subplots((len(epsilons), len(lambdas)))
len(axs)
